# Random Forest(1)

의사결정 트리를 사용하는 가장 대표적인 배깅 모델

의사결정 트리의 단점(과적합이 자주 발생)을 보완하고 장점은 유지.

최근 XGBoost, LightGBM, CatBoost와 함께 주목받는 알고리즘 중 하나.

> 분류 : RandomForestClassifier
> 회귀 : RandomForestRegression

## #01. 패키지

In [1]:
import warnings
warnings.filterwarnings('ignore')

from matplotlib import pyplot as plt
import seaborn as sb
from pandas import read_excel, DataFrame

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import accuracy_score

# 데이터 불균형 해소를 위한 smpling 패키지
from imblearn.under_sampling import RandomUnderSampler 
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

## #02. 데이터

kaggle의 분류 예제 실습용 가상 데이터

> https://www.kaggle.com/competitions/otto-group-product-classification-challenge/data

| 필드이름 | 설명 |
| -- | -- |
| target | 타겟(종속)변수 `(Class_1~Class_9)` |
| feat_1 ~ feat_93 | 설명(독립)변수 |

In [2]:
origin = read_excel("https://data.hossam.kr/G02/otto_train.xlsx")
print(origin.info())
origin.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61878 entries, 0 to 61877
Data columns (total 94 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   feat_1   61878 non-null  int64 
 1   feat_2   61878 non-null  int64 
 2   feat_3   61878 non-null  int64 
 3   feat_4   61878 non-null  int64 
 4   feat_5   61878 non-null  int64 
 5   feat_6   61878 non-null  int64 
 6   feat_7   61878 non-null  int64 
 7   feat_8   61878 non-null  int64 
 8   feat_9   61878 non-null  int64 
 9   feat_10  61878 non-null  int64 
 10  feat_11  61878 non-null  int64 
 11  feat_12  61878 non-null  int64 
 12  feat_13  61878 non-null  int64 
 13  feat_14  61878 non-null  int64 
 14  feat_15  61878 non-null  int64 
 15  feat_16  61878 non-null  int64 
 16  feat_17  61878 non-null  int64 
 17  feat_18  61878 non-null  int64 
 18  feat_19  61878 non-null  int64 
 19  feat_20  61878 non-null  int64 
 20  feat_21  61878 non-null  int64 
 21  feat_22  61878 non-null  int64 
 22

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,1,0,0,1,6,1,5,0,0,1,...,0,1,2,0,0,0,0,0,0,Class_1
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


## #02. 데이터 전처리

### 타겟(종속)변수 라벨링

In [3]:
origin['target'].value_counts().sort_index()

target
Class_1     1929
Class_2    16122
Class_3     8004
Class_4     2691
Class_5     2739
Class_6    14135
Class_7     2839
Class_8     8464
Class_9     4955
Name: count, dtype: int64

In [4]:
origin['target'] = origin['target'].map({
    "Class_1":0,
    "Class_2":1,
    "Class_3":2,
    "Class_4":3,
    "Class_5":4,
    "Class_6":5,
    "Class_7":6,
    "Class_8":7,
    "Class_9":8,
})

origin['target'].value_counts()

target
1    16122
5    14135
7     8464
2     8004
8     4955
6     2839
4     2739
3     2691
0     1929
Name: count, dtype: int64

> 제대로된 분류를 위해서는 데이터가 비슷한 비율로 존재해야 하는 만큼 데이터 불균형을 해결할 필요가 있음.

즉, 전처리 과정에서 비슷한 규모로 데이터를 가질 수 있도록 샘플링 과정이 필요.

사실 가장 좋은 해결 방법은 유의미한 데이터를 더 수집, 추가하는 것

### 독립/종속변수 분리

독립/종속변수 분리 후 데이터 불균형을 해소하는 것이 적합

In [5]:
origin.columns

Index(['feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6', 'feat_7',
       'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12', 'feat_13',
       'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19',
       'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25',
       'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30', 'feat_31',
       'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36', 'feat_37',
       'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42', 'feat_43',
       'feat_44', 'feat_45', 'feat_46', 'feat_47', 'feat_48', 'feat_49',
       'feat_50', 'feat_51', 'feat_52', 'feat_53', 'feat_54', 'feat_55',
       'feat_56', 'feat_57', 'feat_58', 'feat_59', 'feat_60', 'feat_61',
       'feat_62', 'feat_63', 'feat_64', 'feat_65', 'feat_66', 'feat_67',
       'feat_68', 'feat_69', 'feat_70', 'feat_71', 'feat_72', 'feat_73',
       'feat_74', 'feat_75', 'feat_76', 'feat_77', 'feat_78', 'feat_79',
       'feat_80', 'feat_81', 'feat_82', 'feat_83'

In [6]:
x = origin.drop('target', axis=1)
y = origin['target']
x.shape, y.shape

((61878, 93), (61878,))

### 데이터 표준화

> 여기서는 생략 가능

In [7]:
# scaler = StandardScaler()
# std_x = scaler.fit_transform(x)
# std_x[:1]

### 훈련/검증 데이터 분리

1. 기존 코드
2. Under Sampling 방식으로 데이터 불균형을 개선한 x, y 데이터 사용
3. Under Sampling 방식으로 데이터 불균형을 개선한 x, y 데이터 사용

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=777)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((43314, 93), (18564, 93), (43314,), (18564,))

### 데이터 불균형 해소

#### 1. Under Sampling 방식 - Random Under Sampler

많은 비율을 차지하는 다수 집단에서 일부만 샘플링하는 방식.

소수 집단의 데이터가 어느 정도 확보 되었다고 여겨질 때, 다수 집단의 데이터를 줄여서 균형을 맞춘다.

> 다수 집단의 유의미한 데이터를 손실할 수 있다는 단점이 존재.

##### `sampling_strategy 파라미터`

sampling_strategy : 2진 분류일 경우 실수로 설정 가능

| 값 | 설명 |
| -- | -- |
| `majority` | `다수 클래스만` 다시 샘플링 |
| `not majority` | `다수 아님` : 다수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `not minority` | `소수 아님` : 소수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `all` | `모든 클래스`를 다시 샘플링 |
| `auto` | 자동 처리 |

In [9]:
# 모델 생성
undersampler = RandomUnderSampler(sampling_strategy='majority', random_state=777)
# 학습 진행
# 독립변수를 표준화 했다면 표준화한 독립변수를 삽입
x_under, y_under = undersampler.fit_resample(x,y)
print(x_under.shape, y_under.shape)

# Under Sampling 결과 확인
y_under.value_counts().sort_index()

(47685, 93) (47685,)


target
0     1929
1     1929
2     8004
3     2691
4     2739
5    14135
6     2839
7     8464
8     4955
Name: count, dtype: int64

#### 2. Over Sampling 방식 - Random Over Sampler

소수 집단에서 복원 추출을 수행하는 방식.

Under Sampling처럼 데이터 중 일부를 취하는 것은 아니기 때문에 데이터 손실은 발생하지 않지만, 동일한 데이터를 여러 번 학습 데이터에 포함시키므로 학습 정확도는 높지만 과적합 리스크가 존재.

##### `sampling_strategy 파라미터`

sampling_strategy : 2진 분류일 경우 실수로 설정 가능

| 값 | 설명 |
| -- | -- |
| `minority` | `소수 클래스만` 다시 샘플링 |
| `not majority` | `다수 아님` : 다수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `not minority` | `소수 아님` : 소수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `all` | `모든 클래스`를 다시 샘플링 |
| `auto` | 자동 처리 |

In [10]:
# 모델 생성
oversampler = RandomOverSampler(sampling_strategy='minority', random_state=777)
# 학습 진행
x_over, y_over = oversampler.fit_resample(x,y)
print(x_over.shape, y_over.shape)

# Over Sampling 결과 확인
y_over.value_counts().sort_index()

(76071, 93) (76071,)


target
0    16122
1    16122
2     8004
3     2691
4     2739
5    14135
6     2839
7     8464
8     4955
Name: count, dtype: int64

#### 3. Over Sampling - SMOTE

소수 집단의 데이터를 바탕으로 새로운 데이터를 생성.

단순히 소수 집단의 데이터를 복원 추출하는 것이 아니라 소수 집단 데이터를 분석해 어떤 특징이 있는지 살피고 그와 유사한 패턴을 가지는 가짜 데이터를 생성.

##### `sampling_strategy 파라미터`

sampling_strategy : 2진 분류일 경우 실수로 설정 가능

| 값 | 설명 |
| -- | -- |
| `minority` | `소수 클래스만` 다시 샘플링 |
| `not majority` | `다수 아님` : 다수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `not minority` | `소수 아님` : 소수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `all` | `모든 클래스`를 다시 샘플링 |
| `auto` | 자동 처리 |

혹은 실수 타입으로 설정할 경우 샘플 수의 비율을 의미

##### `k_neighbors 파라미터 (int)`

합성 샘플을 생성하는데 사용할 샘플의 가장 가까운 이웃 수 (기본값=5)

In [11]:
# 모델 생성
smote_sampler = SMOTE(sampling_strategy='minority', random_state=777)
# 학습 진행
x_sm, y_sm = smote_sampler.fit_resample(x,y)
print(x_sm.shape, y_sm.shape)

# Over Sampling 결과 확인
y_sm.value_counts().sort_index()

(76071, 93) (76071,)


target
0    16122
1    16122
2     8004
3     2691
4     2739
5    14135
6     2839
7     8464
8     4955
Name: count, dtype: int64

## #03. 랜덤 포레스트 모델 적합

### 단일 모델 만들기

In [13]:
# 모델 생성
# rfc = RandomForestClassifier(n_estimators=20, max_depth=5, random_state=777)
# rfc = RandomForestClassifier(n_estimators=50, max_depth=30, random_state=777)
# rfc = RandomForestClassifier(n_estimators=100, max_depth=50, random_state=777)
rfc = RandomForestClassifier(n_estimators=100, max_depth=100, random_state=777)

# # 학습(원본 데이터)
# rfc.fit(x_train, y_train)
# print(f"원본 데이터의 훈련 정확도 : {rfc.score(x_train, y_train)}")

# # 학습(언더샘플링 데이터)
# rfc.fit(x_under, y_under)
# print(f"언더샘플링의 훈련 정확도 : {rfc.score(x_train, y_train)}")

# # 학습(오버샘플링 데이터)
# rfc.fit(x_over, y_over)
# print(f"오버샘플링의 훈련 정확도 : {rfc.score(x_over, y_over)}")

# 학습(SMOTE 데이터)
rfc.fit(x_sm, y_sm)
print(f"SMOTE의 훈련 정확도 : {rfc.score(x_sm, y_sm)}")

# 테스트 정확도
print(f"테스트 정확도 : {rfc.score(x_test, y_test)}")

SMOTE의 훈련 정확도 : 0.9999342719301705
테스트 정확도 : 0.9999461322990735


> 모델 생성 단계에서 n_estimators, max_depth를 높이면 시간은 걸리지만 성능을 향상 시킬 가능성이 있음.

100% 성능 향상은 기대하기 어려운 만큼 sampling이 필요

### 하이퍼 파라미터 튜닝

> GridSearchCV

`cv` : 쪼개는 단위

`n_jobs` : 실행할 병렬 작업의 수. CPU의 프로세스 수만큼 설정 가능. -1은 모든 프로세서를 사용함을 의미.

In [14]:
# 모델 생성
rfc = RandomForestClassifier(random_state=777)

# 사용할 파라미터 설정
params = {
    'random_state':[20,50,100,100],
    "max_depth":[5,30,50,100]
}

In [15]:
# 모델 생성
grid = GridSearchCV(rfc, param_grid=params, cv=5, n_jobs=-1)
# 학습
grid.fit(x_train, y_train)

print("최적의 하이퍼 파라미터 :",grid.best_params_)
print("최적 훈련 정확도 :",grid.best_score_)

# 예측 값
y_pred = grid.best_estimator_.predict(x_test)
print("최대 검증 정확도 :", accuracy_score(y_test, y_pred))

# 학습 결과 시각화
result_df = DataFrame(grid.cv_results_['params'])
# 평균 값 도출
result_df['mean_test_score'] = grid.cv_results_['mean_test_score']
# 정렬
result_df.sort_values(by='mean_test_score', ascending=False)

최적의 하이퍼 파라미터 : {'max_depth': 100, 'random_state': 50}
최적 훈련 정확도 : 0.8018424335770569
최대 검증 정확도 : 0.8044602456367163


,max_depth,random_state,mean_test_score
13,100,50,0.801842
14,100,100,0.800642
15,100,100,0.800642
8,50,20,0.800619
10,50,100,0.799741
11,50,100,0.799741
9,50,50,0.799718
12,100,20,0.799372
6,30,100,0.796024
7,30,100,0.796024
